In [40]:
#encode data for test
from urllib.parse import quote
print(quote("%25253Cscript%25253Ealert%252528%252522hello%252522%252529%25253C/script%25253E"))

%2525253Cscript%2525253Ealert%25252528%25252522hello%25252522%25252529%2525253C/script%2525253E


In [6]:
from urllib.parse import unquote #This used to decode , encode string data

#For big string input
#log_data = ["2025-12-31T05:03:37.934594Z,127.0.0.1:40924,GET,http://192.168.122.170/dvwa/vulnerabilities/xss_r/?name=%3Cscript%3Ealert%28%22hekk%22%29%3C%2Fscript%3E,/dvwa/vulnerabilities/xss_r/?name=%3Cscript%3Ealert%28%22hekk%22%29%3C%2Fscript%3E,XSS,0,0.0,0"]

# Danger signs for Command Injection

cmd_sign = [";", "&&", "||", "|", "`", "$("]

#This used to read List as String so urllib are read only String not List
#raw_text = log_data[0]

raw_text = input("Enter text to check:")


#Decode the text (converts %3B to ;,etc.)
decoded_input = unquote(raw_text)

#Check for attacks in given string
detected = False
for sign in cmd_sign:
    if sign in decoded_input:
        print(f"DANGER: Command Injection character '{sign}' found")
        detected = True
        break

if not detected:
    print(f"No Command Injection detected.")
    print(f"Decoded result: {decoded_input}")

Enter text to check:<scr<script>ipt>
No Command Injection detected.
Decoded result: <scr<script>ipt>


In [ ]:
OTHER ATTACKS:

In [55]:
import re
from urllib.parse import unquote_plus

def security_scanner():
    # settings
    sqli = [" or 1=1", "union select", "sleep(", "benchmark(", "--", "order by","union"]
    cmd = [";", "&&", "||", "|", "`", "$("]
    path = ["../", "..\\", "php://", "file://", "etc/passwd"]
    xss = ["<script", "javascript:", "onerror=", "onload="]
    nosql = ["$gt", "$ne", "$where", "$regex"]
    
    # Critical Server Attacks
    server_attack = [
    "{{", "${", "<%=",                # SSTI
    "<!entity", "<!doctype",          # XXE
    "%0d%0a", "\\r\\n",               # CRLF
    "__proto__", "constructor.prot",  # Prototype Pollution
    "ac ed 00 05"   # Deserialization (Java)
    ]
    
    
    blocked_ext = [".php", ".py", ".exe", ".jsp", ".sh", "js", ".bash", ".c", ".asm", ".java", ".class"]
    allowed_ext = [".png", ".jpg", ".jpeg", ".gif", ".pdf", ".txt", "text"]

    raw_data = input("enter request: ")
    #raw_data input
    #full decoder if multi codede payloads founds
    def full_decode(raw_text1,max_loop=10):
        previous=None
        current=raw_text1
        for _ in range(max_loop):
            if previous==current:
                break
            previous=current
            current=unquote_plus(current)
        return current
    
    
    text1=full_decode(raw_data)
    text=text1.lower()
    print(text)
    #final text
    status = "normal"

    if 'filename="' in raw_data:
        matches = re.findall(r'filename="([^"]+)"', raw_data, re.IGNORECASE)
        
        found_file_issue = False
        for fname in matches:
            fname = fname.lower()

            # block dangerous extensions
            for ext in blocked_ext:
                if fname.endswith(ext):
                    status = "file_upload_abuse"
                    found_file_issue = True
                    break

            # block double extensions
            if not found_file_issue and fname.count(".") >= 2:
                status = "file_upload_abuse"
                found_file_issue = True

            # allow safe extensions
            if not found_file_issue:
                safe = False
                for ext in allowed_ext:
                    if fname.endswith(ext):
                        status = "normal"
                        safe = True
                        break
                
                # unknown extension -> block
                if not safe:
                    status = "file_upload_abuse"
                    found_file_issue = True
            
            if found_file_issue:
                break

    # 2. non-file attack checks
    if status == "normal":
        found = []
        if any(s in text for s in sqli): found.append("sql_injection")
        if any(s in text for s in cmd): found.append("command_injection")
        if any(s in text for s in path): found.append("path_traversal")
        if any(s in text for s in xss): found.append("xss")
        if any(s in text for s in nosql): found.append("nosql_injection")
        if any(s in text for s in server_attack): found.append("server_attack")
        
        if found:
            status = "|".join(found)

    # final result
    if status != "normal":
        print(f"block: {status}")
    else:
        print("normal")

security_scanner()

enter request: %2526lt%253BScRiPt%2526gt%253Ba%2526lert%25281%2529%2526lt%253B%252Fscript%2526gt
&lt;script&gt;a&lert(1)&lt;/script&gt
block: command_injection


In [ ]:

SQL:
    2026-01-04T10:00:00Z,127.0.0.1,GET,http://site.com/login?user=admin' or 1=1 --,/login,0,0.0
    
COMMAND:
    2026-01-04T10:05:00Z,127.0.0.1,POST,http://site.com/exec,/exec,; cat /etc/passwd,20,4.5

XSS:
    2026-01-04T10:10:00Z,127.0.0.1,GET,http://site.com/search?q=<script>alert('xss')</script>,/search,0,0.0

FILE UPLOAD:
    2026-01-04T10:15:00Z,127.0.0.1,POST,http://site.com/upload,/upload,Content-Disposition: form-data; name="file"; filename="shell.php",500,7.2
    
FILE UPLOAD DUBLE .png.exe:
    2026-01-04T10:20:00Z,127.0.0.1,POST,http://site.com/upload,/upload,Content-Disposition: form-data; name="file"; filename="image.jpg.exe",1200,6.8
    
PATH TRAVEL /LFI:
    2026-01-04T10:25:00Z,127.0.0.1,GET,http://site.com/view?file=../../../../etc/passwd,/view,0,0.0

NOSQL:
    2026-01-04T10:30:00Z,127.0.0.1,POST,http://site.com/api/login,/api/login,{"user": {"$ne": null}, "pass": {"$gt": ""}},85,5.1

COMBINED ATTACK:
    2026-01-04T10:35:00Z,127.0.0.1,POST,http://site.com/test,/test,user=admin'--&cmd=;ls -la,50,4.9

NORMAL:
    2026-01-04T10:40:00Z,127.0.0.1,GET,https://google.com/search?q=weather,/search,0,0.0
    
NOW WE CAN PROTECT TOTAL 9 ATTACKS NOW DDOS ALSO BUT AS LIMITED ONLY


In [1]:
# 1.SIGNATURES (SQLI, CMD, PATH,XSS,NOSQL,FILE INJECTION)
sqli = [" or 1=1", "union select", "sleep(", "benchmark(", "--", "order by"]
cmd = [";", "&&", "||", "|", "`", "$("]
path = ["../", "..\\", "php://", "file://", "etc/passwd", "boot.ini"]
xss = ["<script", "javascript:", "onerror=", "onload=", "alert("]
nosql = ["$gt", "$ne", "$where", "$regex"]

# Critical Server Attacks
server_attack = [
"{{", "${", "<%=",                # SSTI
"<!entity", "<!doctype",          # XXE
"%0d%0a", "\\r\\n",               # CRLF
"__proto__", "constructor.prot",  # Prototype Pollution
"o:", "a:", "s:", "ac ed 00 05"   # Deserialization (PHP/Java)
]

blocked_ext = [".php", ".py", ".exe", ".jsp", ".sh", "js", ".bash", ".c", ".asm", ".java", ".class"]
allowed_ext = [".png", ".jpg", ".jpeg", ".gif", ".pdf", ".txt", "text"]

enter request: sdsjdjsjkdskjdsd:sdsd
normal (entropy: 2.04)


In [16]:
#decode data to find out how many time encode what is atual data
from urllib.parse import unquote
print(unquote("al%65rt(1)"))

alert(1)


In [59]:
from urllib.parse import unquote_plus

def fully_decode(raw_data, max_loops=10):
    previous = None
    current = raw_data

    for _ in range(max_loops):
        if previous == current:
            break
        previous = current
        current = unquote_plus(current)
        print(_)

    return current

raw_data = "%2526lt%253BScRiPt%2526gt%253Ba%2526lert%25281%2529%2526lt%253B%252Fscript%2526gt"

decoded = fully_decode(raw_data)
print(decoded.lower())


0
1
2
&lt;script&gt;a&lert(1)&lt;/script&gt


In [67]:
#DECODE ALL TYPES NOT ONE

BY DECODE ALL THINGS NOT ONE


In [83]:
import re
import html
import unicodedata
from urllib.parse import unquote_plus


def security_scanner():
    # settings
    sqli = [" or 1=1", "union select", "sleep(", "benchmark(", "--", "order by", "union","group by", "having", "extractvalue", "updatexml"]
    cmd = [";", "&&", "||", "|", "`", "$(","&>", "<&", "cmd.exe", "/bin/sh"]
    path = ["../", "..\\", "php://", "file://", "etc/passwd"]
    xss = ["<script", "javascript:", "onerror=", "onload=","onmouseover=", "onfocus=", "<svg", "<iframe"]
    nosql = ["$gt", "$ne", "$where", "$regex"]

    # Critical Server Attacks
    server_attack = [
        "{{", "${", "<%=",                # SSTI
        "<!entity", "<!doctype",          # XXE
        "%0d%0a", "\\r\\n",               # CRLF
        "__proto__", "constructor.prot",  # Prototype Pollution
        "ac ed 00 05"                     # Deserialization (Java)
    ]

    blocked_ext = [".php", ".py", ".exe", ".jsp", ".sh", "js", ".bash", ".c", ".asm", ".java", ".class"]
    allowed_ext = [".png", ".jpg", ".jpeg", ".gif", ".pdf", ".txt", "text"]

    raw_data = input("enter request: ")

    #FULL DECODER (IF MULTI ENCODED PAYLOADS)
    def full_decode(raw_text1, max_loop=10):
        previous = None
        current = raw_text1
        for _ in range(max_loop):
            if previous == current:
                break
            previous = current
            current = unquote_plus(current)
        return current

    #NORMALIZATION
    def normalize(value: str) -> str:
        #URL decode (repeat until stable)
        value = full_decode(value)

        #HTML entity decode
        value = html.unescape(value)

        #Unicode normalize (NFKC)
        value = unicodedata.normalize("NFKC", value)

        #Lowercase
        value = value.lower()

        #Remove comments (SQL / JS)
        value = re.sub(r"/\*.*?\*/", "", value, flags=re.DOTALL) 
        value = re.sub(r"--.*?$", "", value, flags=re.MULTILINE)
        value = re.sub(r"#.*?$", "", value, flags=re.MULTILINE)
            # Preserve CRLF for detection
        value = value.replace("\r\n", "\\r\\n")
        value = value.replace("\n", "\\n").replace("\r", "\\r")

        #Collapse whitespace (tabs, newlines, spaces)
        value = re.sub(r"\s+", " ", value).strip()

        return value

    #APPLY NORMALIZATION
    text = normalize(raw_data)
    print(text)

    status = "normal"

    # FILE UPLOAD CHECK
    if 'filename="' in raw_data:
        matches = re.findall(r'filename="([^"]+)"', raw_data, re.IGNORECASE)

        found_file_issue = False
        for fname in matches:
            fname = fname.lower()

            for ext in blocked_ext:
                if fname.endswith(ext):
                    status = "file_upload_abuse"
                    found_file_issue = True
                    break

            if not found_file_issue and fname.count(".") >= 2:
                status = "file_upload_abuse"
                found_file_issue = True

            if not found_file_issue:
                safe = False
                for ext in allowed_ext:
                    if fname.endswith(ext):
                        status = "normal"
                        safe = True
                        break

                if not safe:
                    status = "file_upload_abuse"
                    found_file_issue = True

            if found_file_issue:
                break

    if status == "normal":
        found = []
        if any(s in text for s in sqli): found.append("sql_injection")
        if any(s in text for s in cmd): found.append("command_injection")
        if any(s in text for s in path): found.append("path_traversal")
        if any(s in text for s in xss): found.append("xss")
        if any(s in text for s in nosql): found.append("nosql_injection")
        if any(s in text for s in server_attack): found.append("server_attack")

        if found:
            status = "|".join(found)

    # ===== FINAL RESULT =====
    if status != "normal":
        print(f"block: {status}")
    else:
        print("normal")


security_scanner()


enter request: %0d%0aSet-Cookie:test=1
\r\nset-cookie:test=1
block: server_attack


In [ ]:
#BUG IN CODE  %0d%0aSet-Cookie:test=1= this take as \r\n

In [ ]:
sql injection 5-6 core type
cmd injection 4-5 core type
path           4
xss             4
file upload     4
nosql           4
SSTI
XXE
CRLF injection
Prototype pollution
Java deserializati
total 30+ 

In [80]:
def normalize(text):
    if not text:
        return ""

    #URL decode (IF MULTI ENCODED PAYLOADS)
    previous = None
    current = text
    for _ in range(5):
        if previous == current:
            break
        previous = current
        current = unquote_plus(current)

    text = current

    #HTML entity decode
    text = html.unescape(text)

    #Unicode normalization (NFKC)
    text = unicodedata.normalize("NFKC", text)

    #Lowercase
    text = text.lower()

    #Remove comments (SQL / JS)
    text = re.sub(r"/\*.*?\*/", "", text, flags=re.DOTALL)
    text = re.sub(r"--.*?$", "", text, flags=re.MULTILINE)
    text = re.sub(r"#.*?$", "", text, flags=re.MULTILINE)
    # Preserve CRLF for detection
    text = text.replace("\r\n", "\\r\\n")
    text = text.replace("\n", "\\n").replace("\r", "\\r")


    #Collapse whitespace (tabs, newlines, spaces)
    text = re.sub(r"\s+", " ", text)

    return text.strip()
text=input("enter text")
print("output is")
print(normalize(text))

enter text%0d%0aSet-Cookie:test=1
output is
\r\nset-cookie:test=1


In [ ]:
# ENTROPY =how random / unpredictable string
#Low entropy → predictable, human text
#High entropy → encoded, encrypted, obfuscated, or machine-generated

In [115]:
import math
def shannon_entropy(s):
    if not s:
        return 0.0
    freq = {}
    for c in s:
        freq[c] = freq.get(c, 0) + 1
    probs = [v / len(s) for v in freq.values()]
    return -sum(p * math.log2(p) for p in probs if p > 0)
string=input("enter string")
# print(shannon_entropy(string))
per=shannon_entropy(string)+10
if shannon_entropy(string) >= 1:
    per=shannon_entropy(string)*10
print(f"{per} percentage attack string")

enter stringrutik
23.21928094887362 percentage attack string
